In [1]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LogColorMapper, LogTicker, ColorBar, NumeralTickFormatter
from bokeh.palettes import brewer
from bokeh.models import HoverTool
from bokeh.io import export_svgs
import geopandas as gpd
import pandas as pd
import json

#Load map file, rename its columns and drop Antarctica
shapefile = 'ne_110m_admin_0_countries.shp'
gdf = gpd.read_file(shapefile)[['ADMIN' , 'ADM0_A3', 'geometry']]
gdf.columns = ['country', 'country_code', 'geometry']
gdf = gdf.drop(gdf.index[159])

#Load our input file and give names to columns
datafile = 'slash_24_map_20190718.txt'
df = pd.read_csv(datafile, names = ['Country', 'Count'])

#Include in map countries for which we have no data to avoid holes
merged = gdf.merge(df, left_on = 'country_code', right_on = 'Country', how = 'left')
merged.fillna('No data', inplace = True)

#Some json stuff used by bokeh later ...
merged_json = json.loads(merged.to_json())
json_data = json.dumps(merged_json)

#Plotting the map
geosource = GeoJSONDataSource(geojson = json_data)
palette = brewer['YlGnBu'][9]
palette = palette[::-1]

color_mapper = LogColorMapper(palette = palette, low = 1, high = 200000, nan_color = '#d9d9d9')
tick_labels = {'1000': '1 000', '10000':'10 000', '100000':'100 000+'}

hover = HoverTool(tooltips = [ ('Country','@country'),('Number of resolvers', '@Count')])
color_bar = ColorBar(color_mapper=color_mapper, ticker=LogTicker(), label_standoff=9,width = 800, height = 30, major_label_overrides = tick_labels, border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_text_font_size='25px')
color_bar.formatter.use_scientific = False
p = figure(plot_height = 700 ,plot_width = 1200, toolbar_location = None, tools = [hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

p.patches('xs','ys', source = geosource,fill_color = {'field' :'Count', 'transform' : color_mapper}, line_color = 'black', line_width = 0.25, fill_alpha = 1)
p.add_layout(color_bar, 'below')

#Output the map in the notebook
output_notebook()

#Save as html
output_file("slash_24_map_20190718.html")

#Export as svg
p.output_backend = "svg"
export_svgs(p, filename="slash_24_map_20190718.svg")

show(p)

Loading BokehJS ...